# Testing MCTS with Various Models
Here we will test MCTS first with the DKT model

In [1]:
#%load_ext autoreload
#%autoreload 2
#%reload_ext autoreload

from concept_dependency_graph import ConceptDependencyGraph
import data_generator as dg
from student import *
import simple_mdp as sm

In [2]:
n_concepts = 4
use_student2 = True
learn_prob = 0.15
n_students = 100000
seqlen = 6
policy = 'random'
filename = 'test{}-n{}-l{}-lp{}-{}.pickle'.format('2' if use_student2 else '', n_students, seqlen, int(learn_prob*100), policy)
#concept_tree = sm.create_custom_dependency()
concept_tree = ConceptDependencyGraph()
concept_tree.init_default_tree(n_concepts)
if not use_student2:
    test_student = Student(n=n_concepts,p_trans_satisfied=learn_prob, p_trans_not_satisfied=0.0, p_get_ex_correct_if_concepts_learned=1.0)
else:
    test_student = Student2(n_concepts)
print(filename)

test2-n100000-l6-lp15-random.pickle


In [3]:
print ("Initializing synthetic data sets...")
dg.generate_data(concept_tree, student=test_student, n_students=n_students, seqlen=seqlen, policy=policy, filename="{}{}".format(dg.SYN_DATA_DIR, filename))
print ("Data generation completed. ")

Initializing synthetic data sets...
Generating data for 100000 students with behavior policy random and sequence length 7.
Data generation completed. 


In [3]:
import dynamics_model_class as dmc
import numpy as np
import dataset_utils

In [4]:
# test_model hidden=16
# test_model_small hidden=5
# test_model_mid hidden=10
dmodel = dmc.DynamicsModel(model_id="test2_model_mid", timesteps=seqlen, load_checkpoint=False)

Loading RNN dynamics model...
Directory path for tensorboard summaries: ../tensorboard_logs/test2_model_mid/
Checkpoint directory path: ../checkpoints/test2_model_mid/
Model loaded.


In [5]:
# load toy data
data = dataset_utils.load_data(filename='{}{}'.format(dg.SYN_DATA_DIR, filename))
input_data_, output_mask_, target_data_ = dataset_utils.preprocess_data_for_rnn(data)

In [6]:
train_data = (input_data_[:,:,:], output_mask_[:,:,:], target_data_[:,:,:])
print(input_data_.shape)
print(output_mask_.shape)
print(target_data_.shape)

(100000, 6, 8)
(100000, 6, 4)
(100000, 6, 4)


In [7]:
dmodel.train(train_data, n_epoch=5, load_checkpoint=False)

Training Step: 7034  | total loss: 0.00153 | time: 19.774s
| Adam | epoch: 005 | loss: 0.00153 -- iter: 89984/90000
Training Step: 7035  | total loss: 0.00151 | time: 20.950s
| Adam | epoch: 005 | loss: 0.00151 | val_loss: 0.00155 -- iter: 90000/90000
--
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'


In [70]:
preds = dmodel.predict(input_data_[:1,:2, :])
print len(preds)
print len(preds[0])
print preds[0]

1
6
[[0.7310575246810913, 0.0005168265197426081, 0.0005419104709289968, 1.4160171701860236e-07], [0.8807966709136963, 0.0003708214790094644, 0.880794107913971, 4.6009071752450836e-07], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]


In [53]:
rnnsim = dmc.RnnStudentSim(dmodel)

In [54]:
from mcts import *
actix = 0
actvec = np.zeros((n_concepts,))
actvec[actix] = 1
act = StudentAction(actix, actvec)
print rnnsim.sequence
print rnnsim.sample_observations()

[]
None


In [55]:
rnnsim.advance_simulator(act, 1)


In [42]:
print rnnsim.sequence
print rnnsim.sample_observations()
print rnnsim.sample_reward()

[array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])]
[0.7301660180091858, 0.020661335438489914, 0.030488919466733932, 0.00025640291278250515]
0.781572675827
